In [ ]:
from models.mysql_db import generate_symbols
from models.sqliteModels import insert_price_data
from src.pricehistory import price_history
from loguru import logger
import os.path
import time

In [ ]:
log_path = str(os.path.curdir) + '/logs/'
base_fmt = "[{time:YYYY-MM-DD at HH:mm:ss} - {level}] | [{name} - <lvl>{message}</lvl>]"
logger.add(log_path+"pricehistory.log", rotation="20 MB")
